In [5]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

import cv2

import time

In [7]:
from web_interaction.main import open_game
from game.game import ProductOwnerGame
import image_parser

In [3]:
driver = open_game()

In [ ]:
time.sleep(10)

In [4]:
iframe = driver.find_element(by=By.ID, value='game_drop')
height = iframe.rect["height"]  # 540
width = iframe.rect["width"]  # 960

In [5]:
iframe.click()
iframe.click()

In [6]:
# type name
ActionChains(driver).move_to_element_with_offset(
    iframe, 0, int(0.1 * height)
).click().send_keys("selenium").perform()

In [7]:
# start game
ActionChains(driver).move_to_element_with_offset(
    iframe, 0, int(0.2 * height)
).click().perform()

In [8]:
# skip tutorial
ActionChains(driver).move_to_element_with_offset(
    iframe, -int(0.35 * width), int(0.4 * height)
).click().perform()

In [9]:
# turn off sprint animation
ActionChains(driver).move_to_element_with_offset(
    iframe, -int(0.45 * width), -int(0.42 * height)  # move to settings icon
).click().move_to_element_with_offset(
    iframe, int(0.1 * width), int(0.07 * height)  # move to animation checkbox
).click().move_to_element_with_offset(
    iframe, -int(0.45 * width), -int(0.42 * height)  # move to settings icon
).click().perform()

In [10]:
# buy research
ActionChains(driver).move_to_element_with_offset(
    iframe, int(0.3 * width), -int(0.3 * height)
).click().perform()

In [10]:
iframe.screenshot('iframe.png')

True

In [8]:
image = cv2.imread('iframe.png')
meta_info = image[7:83, 57:932]

In [9]:
game = ProductOwnerGame()
context = game.context

In [23]:
context.is_new_game = False

In [10]:
current_sprint = image_parser.get_sprint_number(meta_info)
context.current_sprint = current_sprint
current_sprint

4.0

In [15]:
available_developers_count = 2
current_rooms_counter = 1
blank_sprint_counter = 0

In [16]:
current_sprint_hours = image_parser.get_current_sprint_hours(image)
current_sprint_hours

0.0

In [17]:
user_stories = image_parser.get_user_stories(image)
user_stories

[(array([115, 188,  30], dtype=uint8), 0.055, 2.0)]

In [18]:
import sys
sys.path.insert(0, '..')
from game.game_colors import ColorStorage


class SingleColorStorage(ColorStorage):
    def __init__(self, color):
        self.color = color
    
    def get_unused_color(self, uc_type):
        return self.color

In [21]:
from game.userstory_card.userstory_card_info import UserStoryCardInfo

for user_story in user_stories:
    color, loyalty, customers = user_story
    color_storage = SingleColorStorage(color)
    game_user_story = UserStoryCardInfo('S', current_sprint, color_storage)
    game_user_story.loyalty = loyalty
    game_user_story.customers_to_bring = customers

    game.userstories.add_us(game_user_story)

In [22]:
money = image_parser.get_game_money(meta_info)
context.set_money(money)

In [24]:
loyalty = image_parser.get_loyalty(meta_info)
context.set_loyalty(loyalty)

In [25]:
context.customers = image_parser.get_customers(meta_info)

In [ ]:
credit = max(300_000 - (current_sprint - 1) * 9_000, 0)
context.credit = credit
credit

In [13]:
iframe.rect

{'height': 540, 'width': 960, 'x': 29.600000381469727, 'y': 20}

In [25]:
driver.quit()